# Instrument and Quality Recognition using spaCy's NER

## Import libraries

In [31]:
import spacy 
from spacy import displacy
import pandas as pd

import numpy
numpy.random.seed(0)

## Download and load spaCy language model

In [32]:
#Download spacy small model
!python -m spacy download en_core_web_sm

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Getting the pipeline component
ner = nlp.get_pipe("ner")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


## Load Doccano labelled dataset

In [34]:
import json

with open('./project_1_dataset.jsonl', 'r') as json_file:
    json_list = list(json_file)

df_dataset = pd.DataFrame()

DATA = []

for json_str in json_list:
    result = json.loads(json_str)

    # Filter out those entries which do NOT have any annotations
    if(bool(result['annotations'])==True):
      # print(result)
      annot_list = []
      for i in range(len(result['annotations'])):
        start_idx = result['annotations'][i]['start_offset']
        end_idx = result['annotations'][i]['end_offset']
        label = "INSTRUMENT" if result['annotations'][i]['label'] == 1 else "QUALITY"
        annot_list.append((start_idx, end_idx, label))

      DATA.append((result['text'], {"entities": annot_list}))


for i in range(5):
  print(DATA[i])

('[Example] I would like to find a dark sounding piano.', {'entities': [(33, 37, 'QUALITY'), (47, 52, 'INSTRUMENT')]})
('[Example] The composition that I am trying to write would really benifit from a thick bass.', {'entities': [(80, 85, 'QUALITY'), (86, 90, 'INSTRUMENT')]})
('[Example] Does anyone know what kind of guitar pedals I could use to generate a thin sound?', {'entities': [(40, 46, 'INSTRUMENT'), (80, 84, 'QUALITY')]})
('"Electric guitar timbre is so malleable that generalizations like this make little sense.  Beyond the instrument itself, every guitarist has personal preferences in amplifiers and a wide variety of stomp box effects.  The overtone series is dependent on the selection of vacuum tubes in the amplifier - the \\"British\\" (Marshall) vs \\"American\\" (Fender) sound. \\n\\nIn this [Bill Frisell concert](https://youtu.be/ifr8IowlQIM) the difference in timbre between violin and electric guitar couldn\'t be more obvious.",', {'entities': [(1, 16, 'INSTRUMENT'), (471

## Prepare TRAIN and TEST data

Randomly pull out 5 segments for test data

In [35]:
import random
random.shuffle(DATA)

# First 5 elements form test data after shuffling
TEST_DATA = DATA[:5]

for text, annotations in TEST_DATA:
  print(text)
  print(annotations)

TRAIN_DATA = DATA[5:len(DATA)]

# for text, annotations in TRAIN_DATA:
#   print(text)
#   print(annotations)

print("\nLength of test data: ", len(TEST_DATA))
print("Lenght of train data: ", len(TRAIN_DATA))


"Almost no instrument produces a pure tone.  The flute is about the closest.  Generally, an instrument produces a load of notes that make up the harmonic series.  These are notes whose frequencies are a whole number multiple of times the base (or fundamental) frequency.  So, when you hear a piano playing 440Hz concert A, you're also hearing a little bit of the A an octave higher at 880Hz, and the E above that at 1320Hz, and the A at 1760Hz, etc.\n\nThe exact mix of these notes is what determines tone or timbre.\n\nYou can see this for yourself.  Download a free spectrum analyser app and sing in to it.  If you whistle, you'll see a sharp peak, pretty close to a pure tone.  Now sing a few different vowels and you'll see a bunch of spikes.  As you keep the note the same but change the vowels, you'll see the lowest not stays in the same place, but the higher tones change in position and amount.",
{'entities': [(33, 42, 'QUALITY'), (49, 55, 'INSTRUMENT'), (292, 298, 'INSTRUMENT'), (669, 678

## Adding labels to the `ner`


In [36]:
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

## Disable pipeline components that is not changed

In [37]:
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

## Train NER

In [38]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

ITERATIONS = 100
DROPOUT = 0.1

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):
  for iteration in range(ITERATIONS):
    # print("Iteration: ", iteration)
    # shufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop = DROPOUT,  # dropout - make it harder to memorise data
                    losses=losses
                )
        # print("Losses", losses)

print("Training finished")

Training finished


In [40]:
for example in TEST_DATA:
  print(example[0])
  doc = nlp(example[0])
  print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

"Almost no instrument produces a pure tone.  The flute is about the closest.  Generally, an instrument produces a load of notes that make up the harmonic series.  These are notes whose frequencies are a whole number multiple of times the base (or fundamental) frequency.  So, when you hear a piano playing 440Hz concert A, you're also hearing a little bit of the A an octave higher at 880Hz, and the E above that at 1320Hz, and the A at 1760Hz, etc.\n\nThe exact mix of these notes is what determines tone or timbre.\n\nYou can see this for yourself.  Download a free spectrum analyser app and sing in to it.  If you whistle, you'll see a sharp peak, pretty close to a pure tone.  Now sing a few different vowels and you'll see a bunch of spikes.  As you keep the note the same but change the vowels, you'll see the lowest not stays in the same place, but the higher tones change in position and amount.",
Entities [('flute', 'INSTRUMENT'), ('sharp', 'QUALITY')]
"What does \"good guitar tone\" mean?

## Display test data and its original tags

In [41]:
for text, annotations in TEST_DATA:
  print(text)
  print(annotations)

"Almost no instrument produces a pure tone.  The flute is about the closest.  Generally, an instrument produces a load of notes that make up the harmonic series.  These are notes whose frequencies are a whole number multiple of times the base (or fundamental) frequency.  So, when you hear a piano playing 440Hz concert A, you're also hearing a little bit of the A an octave higher at 880Hz, and the E above that at 1320Hz, and the A at 1760Hz, etc.\n\nThe exact mix of these notes is what determines tone or timbre.\n\nYou can see this for yourself.  Download a free spectrum analyser app and sing in to it.  If you whistle, you'll see a sharp peak, pretty close to a pure tone.  Now sing a few different vowels and you'll see a bunch of spikes.  As you keep the note the same but change the vowels, you'll see the lowest not stays in the same place, but the higher tones change in position and amount.",
{'entities': [(33, 42, 'QUALITY'), (49, 55, 'INSTRUMENT'), (292, 298, 'INSTRUMENT'), (669, 678

## Test on custom unseen data

In [42]:
doc = nlp("Give me a bright guitar")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

doc = nlp("Give me a sharp cello")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])


Entities [('bright guitar', 'INSTRUMENT')]
Entities [('sharp', 'QUALITY')]
